![image.png](https://raw.githubusercontent.com/fjvarasc/DSPXI/master/figures/python_logo.png)

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Guided-Lab-:-Collecting-the-Data" data-toc-modified-id="Guided-Lab-:-Collecting-the-Data-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Guided Lab : Collecting the Data</a></span><ul class="toc-item"><li><span><a href="#Combining-Data" data-toc-modified-id="Combining-Data-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Combining Data</a></span></li><li><span><a href="#Using-Categories" data-toc-modified-id="Using-Categories-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Using Categories</a></span></li></ul></li></ul></div>

# Guided Lab : Collecting the Data

Import pandas and numpy

In [ ]:
import pandas as pd
import numpy as np

let's import each of our files and combine them into one file. [Panda's concat and append](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html) can do this for us. Then use append in this example.

The code snippet below will initialize a blank DataFrame then append all of the individual files into the all_data DataFrame.

In [ ]:
all_data = pd.DataFrame()
df1 = pd.read_excel('https://github.com/fjvarasc/DSPXI/blob/master/data/sales-mar-2014.xlsx?raw=true')
df2 = pd.read_excel('https://github.com/fjvarasc/DSPXI/blob/master/data/sales-feb-2014.xlsx?raw=true')
df3 = pd.read_excel('https://github.com/fjvarasc/DSPXI/blob/master/data/sales-jan-2014.xlsx?raw=true')
all_data = all_data.append([df1,df2,df3],ignore_index=True)

In [ ]:
all_data.dtypes

Now we have all the data in our all_data DataFrame. but there is one detail, as we saw before the date field  is not recognized as date, it is not critical but the best practice is to convert the date column to a date time object. we can do so by running the below code.

In [ ]:
all_data['date'] = all_data['date'].astype('datetime64[ns]')
#all_data['date'] = pd.to_datetime(all_data['date']) this will be valid for the same too
all_data.dtypes

You can use describe to look at it and make sure you data looks good.

In [ ]:
all_data.describe()

A lot of this data may not make much sense for this data set but for outr purpose we are interested in the count row to make sure the number of data elements makes sense.

In [ ]:
all_data.head()

## Combining Data

Now that we have all of the data into one DataFrame, we can do any manipulations the DataFrame supports. In this case, the next thing we want to do is read in another file that contains the customer status by account. You can think of this as a company's customer segmentation strategy or some other mechanism for identifying their customers.

First, we read in the data.

In [ ]:
status = pd.read_excel("https://github.com/fjvarasc/DSPXI/blob/master/data/customer-status.xlsx?raw=true")
status

We want to merge this data with our concatenated data set of sales. We use panda's merge function and tell it to do a left join which is similar to Excel's vlookup function.

In [ ]:
all_data_st = pd.merge(all_data, status, how='left')
all_data_st.head()

This looks pretty good but let's look at a specific account.

In [ ]:
all_data_st[all_data_st["account number"]==737550].head()

This account number was not in our status file, so we have a bunch of NaN's. We can decide how we want to handle this situation. For this specific case, let's label all missing accounts as bronze. Use the fillna function to easily accomplish this on the status column.

In [ ]:
all_data_st['status'].fillna('bronze',inplace=True)
all_data_st.head()

Check the data just to make sure we're all good.

In [ ]:
all_data_st[all_data_st["account number"]==737550].head()

Now we have all of the data along with the status column filled in. We can do our normal data manipulations using the full suite of pandas capability.

## Using Categories

One of the relatively new functions in pandas is support for categorical data. From the pandas, [documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html).

"Categoricals are a pandas data type, which correspond to categorical variables in statistics: a variable, which can take on only a limited, and usually fixed, number of possible values (categories; levels in R). Examples are gender, social class, blood types, country affiliations, observation time or ratings via Likert scales."

For our purposes, the status field is a good candidate for a category type.

First, we `typecast` it to a category using [astype](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.astype.html).

In [ ]:
all_data_st["status"] = all_data_st["status"].astype("category")

This doesn't immediately appear to change anything yet.

In [ ]:
all_data_st.head()

Buy you can see that it is a new data type.

In [ ]:
all_data_st.dtypes

Categories get more interesting when you assign order to the categories. Right now, if we call sort on the column, it will sort alphabetically. 

In [ ]:
all_data_st.sort_values(by=["status"]).head()

We use set_categories to tell it the order we want to use for this category object. In this case, we use the Olympic medal ordering.

In [ ]:
 all_data_st["status"].cat.set_categories([ "gold","silver","bronze"],inplace=True)

Now, we can sort it so that gold shows on top.

In [ ]:
all_data_st.sort_values(by=["status"]).head()

In [ ]:
all_data_st["status"].describe()

For instance, if you want to take a quick look at how your top tier customers are performaing compared to the bottom. Use groupby to give us the average of the values.

In [ ]:
all_data_st.groupby(["status"])["quantity","unit price","ext price"].mean()

Of course, you can run multiple aggregation functions on the data to get really useful information 

In [ ]:
all_data_st.groupby(["status"])["quantity","unit price","ext price"].agg([np.sum,np.mean, np.std])

So, what does this tell you? Well, the data is completely random but the first observation is that we sell more units to our bronze customers than gold. Even when you look at the total dollar value associated with bronze vs. gold, it looks backwards.

Maybe we should look at how many bronze customers we have and see what is going on.

What I plan to do is filter out the unique accounts and see how many gold, silver and bronze customers there are.

We've purposely stringing a lot of commands together which is not necessarily best practice but does show how powerful pandas can be.

In [ ]:
all_data_st.drop_duplicates(subset=["account number","name"]).iloc[:,[0,1,7]].groupby(["status"])["name"].count()

Ok. This makes a little more sense. We see that we have 9 bronze customers and only 4 gold customers. That is probably why the volumes are so skewed towards our bronze customers.